# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [47]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory(r'C:\Users\Kartik\Downloads\MLCourse\MLCourse\emails\spam', 'spam'))
data = data.append(dataFrameFromDirectory(r'C:\Users\Kartik\Downloads\MLCourse\MLCourse\emails\ham', 'ham'))


C:\Users\Kartik\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Let's have a look at that DataFrame:

In [48]:
data.head()

,class,message
C:\Users\Kartik\Downloads\MLCourse\MLCourse\emails\spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
C:\Users\Kartik\Downloads\MLCourse\MLCourse\emails\spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
C:\Users\Kartik\Downloads\MLCourse\MLCourse\emails\spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
C:\Users\Kartik\Downloads\MLCourse\MLCourse\emails\spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
C:\Users\Kartik\Downloads\MLCourse\MLCourse\emails\spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [49]:
from sklearn.model_selection import train_test_split

y=data['class']
vectorizer = CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)

counts = vectorizer.fit_transform(X_train['message'].values)

classifier = MultinomialNB()
targets = X_train['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [50]:
examples =X_test['message'].values
example_counts = vectorizer.transform(examples)

predictions = classifier.predict(example_counts)
c=0
for x in range(len(predictions)):
    if(predictions[x]==y[x]):
        c+=1
        
print(c/len(predictions))

0.25333333333333335
